In [1]:

import os
from pathlib import Path

# Set your working directory explicitly
project_dir = Path(r"E:\DeepLearning\Kidney-Disease-Classification")
os.chdir(project_dir)
print("Working directory set to:", os.getcwd())


Working directory set to: E:\DeepLearning\Kidney-Disease-Classification


In [2]:
# Cell 1 — setup (token inline, per your request)
import os
from pathlib import Path



# Use YOUR repo on DagsHub
DAGSHUB_USERNAME = "ACHYUTH1203"
DAGSHUB_REPO     = "Kidney-Disease-Classification"

# MLflow tracking URL for your repo
MLFLOW_TRACKING_URI = f"https://dagshub.com/{DAGSHUB_USERNAME}/{DAGSHUB_REPO}.mlflow"

# === YOUR TOKEN (from the screenshot) ===
TOKEN = "7d82077e1fc014eeaf3b04b0572df2f6d80a8ca9"

# Set env vars for MLflow auth
os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI
os.environ["MLFLOW_TRACKING_USERNAME"] = DAGSHUB_USERNAME
os.environ["MLFLOW_TRACKING_PASSWORD"] = TOKEN

# DagsHub tip: disable MLflow “Logged Models” (unsupported there)
os.environ["MLFLOW_ENABLE_LOGGED_MODELS"] = "false"

print("MLFLOW_TRACKING_URI =", os.environ["MLFLOW_TRACKING_URI"])
print("User =", os.environ["MLFLOW_TRACKING_USERNAME"])


MLFLOW_TRACKING_URI = https://dagshub.com/ACHYUTH1203/Kidney-Disease-Classification.mlflow
User = ACHYUTH1203


In [3]:
from pathlib import Path

project_dir = Path(r"E:\DeepLearning\Kidney-Disease-Classification")
ROOT_DIR = Path.cwd()
CONFIG_FILE_PATH = ROOT_DIR / "config" / "config.yaml"
PARAMS_FILE_PATH = ROOT_DIR / "params.yaml"

In [ ]:
# Cell 2 — config objects
from dataclasses import dataclass
from pathlib import Path
import tensorflow as tf

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config=EvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            training_data=Path("artifacts/data_ingestion/kidney-ct-scan-image"),
            mlflow_uri=os.environ["MLFLOW_TRACKING_URI"],   # <- use the env we set in Cell 1
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE)
        return eval_config


In [5]:
# Cell 3 — evaluation & logging
import mlflow

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.model: tf.keras.Model = None
        self.valid_generator = None
        self.score = None  # (loss, accuracy)

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        if not path.exists():
            raise FileNotFoundError(f"Model file not found at: {path.resolve()}")
        return tf.keras.models.load_model(path)

    def _valid_generator(self):
        datagenerator_kwargs = dict(rescale=1.0/255.0, validation_split=0.30)
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=str(self.config.training_data),
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        return self.valid_generator

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        # ✅ FIX: evaluate the instance model you just loaded
        self.score = self.model.evaluate(self.valid_generator, verbose=1)
        self._save_score()

    def _save_score(self):
        out_dir = Path("artifacts/evaluation")
        out_dir.mkdir(parents=True, exist_ok=True)
        scores = {"loss": float(self.score[0]), "accuracy": float(self.score[1])}
        save_json(path=out_dir / "scores.json", data=scores)

    def log_into_mlflow(self):
        """
        DagsHub-compatible:
        - set TRACKING URI (not registry)
        - log params/metrics
        - log a single-file .keras model as an artifact under Artifacts/model/
        """
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        mlflow.set_experiment("kidney-clf-evaluation")

        with mlflow.start_run() as run:
            run_id = run.info.run_id

            # Params
            mlflow.log_params({
                k: (str(v) if not isinstance(v, (int, float, str, bool)) else v)
                for k, v in self.config.all_params.items()
            })

            # Metrics
            mlflow.log_metrics({
                "loss": float(self.score[0]),
                "accuracy": float(self.score[1]),
            })

            # Single-file Keras model (Keras 3) -> save locally, then upload as artifact
            model_file = Path("artifacts/evaluation/model_export.keras")
            model_file.parent.mkdir(parents=True, exist_ok=True)
            self.model.save(model_file)  # writes one .keras file

            mlflow.log_artifact(str(model_file), artifact_path="model")

            # Optional: add an info file & tags
            model_rel = "model/model_export.keras"
            model_uri = f"runs:/{run_id}/artifacts/{model_rel}"
            mlflow.set_tag("model_artifact_path", model_rel)
            mlflow.set_tag("model_uri", model_uri)
            mlflow.log_dict(
                {"format": "keras", "file": model_rel, "model_uri": model_uri},
                artifact_file="model/model_info.json"
            )

            # Also attach scores.json
            scores_path = Path("artifacts/evaluation/scores.json")
            if scores_path.exists():
                mlflow.log_artifact(str(scores_path), artifact_path="evaluation")

        print("\nLogged model artifact at:", model_rel)
        print("Model URI:", model_uri)


In [6]:

try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-08-13 11:13:21,850: INFO: common: yaml file: E:\DeepLearning\Kidney-Disease-Classification\config\config.yaml loaded successfully]
[2025-08-13 11:13:21,868: INFO: common: yaml file: E:\DeepLearning\Kidney-Disease-Classification\params.yaml loaded successfully]
[2025-08-13 11:13:21,868: INFO: common: created directory at: artifacts]
[2025-08-13 11:13:22,363: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 227 images belonging to 2 classes.


e:\DeepLearning\Kidney-Disease-Classification\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


15/15 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.9031 - loss: 0.1705
[2025-08-13 11:13:43,753: INFO: common: json file saved at: artifacts\evaluation\scores.json]
🏃 View run serious-ray-341 at: https://dagshub.com/ACHYUTH1203/Kidney-Disease-Classification.mlflow/#/experiments/1/runs/4804d5375773479e880c3be35b1dc4db
🧪 View experiment at: https://dagshub.com/ACHYUTH1203/Kidney-Disease-Classification.mlflow/#/experiments/1

Logged model artifact at: model/model_export.keras
Model URI: runs:/4804d5375773479e880c3be35b1dc4db/artifacts/model/model_export.keras
